In [ ]:
import pandas as pd

# -------- 1. Load Excel --------
excel_path = "/.jugdish.xlsx"
xls = pd.ExcelFile(excel_path)
df = xls.parse(xls.sheet_names[0], header=None)

# -------- 2. Extract Metadata (Rows 0–18) --------
metadata_df = df.iloc[0:19].fillna("")
metadata_lines = metadata_df.astype(str).apply(
    lambda row: " | ".join(cell for cell in row if cell.strip() != ''), axis=1
)
metadata_text = "\n".join(metadata_lines)

# -------- 3. Extract Header + Transaction Data --------
header_row = df.iloc[19].dropna().tolist()
data_rows = df.iloc[20:].dropna(how='all').reset_index(drop=True)

# Assign headers
data_rows.columns = header_row + list(data_rows.columns[len(header_row):])
data_rows = data_rows[header_row]

# Parse transaction date and drop bad rows
data_rows["Transaction Date"] = pd.to_datetime(data_rows["Transaction Date"], errors='coerce').dt.date
data_rows = data_rows.dropna(subset=["Transaction Date"])

# -------- 4. Create Chunks in Natural Order of Appearance --------
chunks_for_qdrant = []

# Chunk 0: Metadata only
chunks_for_qdrant.append({
    "id": "chunk_0",
    "text": metadata_text,
    "metadata": {
        "type": "metadata_only",
        "source": "jugdish.xlsx",
        "chunk_index": 0
    }
})

# Loop through transactions row by row, grouping by consecutive date
current_date = None
current_chunk_rows = []
chunk_index = 1

for _, row in data_rows.iterrows():
    row_date = row["Transaction Date"]

    if current_date is None:
        current_date = row_date

    if row_date != current_date:
        # Create and store chunk for the previous date
        chunk_df = pd.DataFrame(current_chunk_rows)
        chunk_text = chunk_df.to_string(index=False, header=True)

        chunks_for_qdrant.append({
            "id": f"chunk_{chunk_index}",
            "text": chunk_text,
            "metadata": {
                "type": "transactions_by_day",
                "transaction_date": str(current_date),
                "source": "jugdish.xlsx",
                "chunk_index": chunk_index
            }
        })

        # Prepare for the new date
        current_chunk_rows = []
        current_date = row_date
        chunk_index += 1

    # Add current row to buffer
    current_chunk_rows.append(row)

# Final chunk (after loop ends)
if current_chunk_rows:
    chunk_df = pd.DataFrame(current_chunk_rows)
    chunk_text = chunk_df.to_string(index=False, header=True)

    chunks_for_qdrant.append({
        "id": f"chunk_{chunk_index}",
        "text": chunk_text,
        "metadata": {
            "type": "transactions_by_day",
            "transaction_date": str(current_date),
            "source": "jugdish.xlsx",
            "chunk_index": chunk_index
        }
    })

# -------- 5. Optional: View a Specific Chunk --------
chunk_id_to_view = "chunk_12"  # Change as needed

for chunk in chunks_for_qdrant:
    if chunk["id"] == chunk_id_to_view:
        print(f"\n=== Chunk ID: {chunk['id']} ===")
        print("--- Metadata ---")
        for key, value in chunk["metadata"].items():
            print(f"{key}: {value}")
        print("\n--- Text Block ---")
        print(chunk["text"])
        break


=== Chunk ID: chunk_12 ===
--- Metadata ---
type: transactions_by_day
transaction_date: 2024-03-07
source: jugdish.xlsx
chunk_index: 12

--- Text Block ---
Branch Name/ IFSC Code                     Transaction ID Transaction Date Transaction Type  Instrument Number                                                                                       Narration  Debit Amount  Credit Amount  Line Balance
 777 - NETBANK, MUMBAI 551460065202407039999990453592349'       2024-03-07           Others       455191079932             UPI-AMANJOT  KAUR DO MAN-8729034444@YBL-SBIN0001529-455191079932-PAYMENT FROM PHONE           499              0       2068.09
 777 - NETBANK, MUMBAI 551460065202407039999990464411939'       2024-03-07           Others       418545363013                UPI-MANPREET  SINGH-MS357403@OKICICI-INDB0000941-418545363013-PAYMENT FROM PHONE           300              0       1768.09
 777 - NETBANK, MUMBAI 551460065202407039999990466413506'       2024-03-07         Transfer  

In [ ]:
import requests

# --- Config ---
JINA_API_KEY = ""
JINA_ENDPOINT = "https://api.jina.ai/v1/embeddings"
EMBEDDING_MODEL = "jina-embeddings-v3"
EMBEDDING_DIM = 1024
EMBEDDING_TASK = "retrieval.passage"
BATCH_SIZE = 32

def generate_jina_dense_embeddings(text_batch):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {JINA_API_KEY}"
    }
    data = {
        "input": text_batch,
        "model": EMBEDDING_MODEL,
        "dimensions": EMBEDDING_DIM,
        "task": EMBEDDING_TASK
    }
    response = requests.post(JINA_ENDPOINT, headers=headers, json=data)
    response.raise_for_status()
    return [item["embedding"] for item in response.json()["data"]]

# --- Extract raw text from chunks ---
chunk_texts = [chunk["text"] for chunk in chunks_for_qdrant]

# --- Generate dense embeddings ---
embedded_chunks = []
for i in range(0, len(chunk_texts), BATCH_SIZE):
    batch_texts = chunk_texts[i:i+BATCH_SIZE]
    batch_embeddings = generate_jina_dense_embeddings(batch_texts)

    for j, embedding in enumerate(batch_embeddings):
        chunk_index = i + j
        embedded_chunks.append({
            "id": chunks_for_qdrant[chunk_index]["id"],
            "text": chunks_for_qdrant[chunk_index]["text"],
            "embedding": embedding,
            "metadata": chunks_for_qdrant[chunk_index]["metadata"]
        })

# --- Optional: Print to verify ---
print(f"Generated embeddings for {len(embedded_chunks)} chunks.")
print("Example:", embedded_chunks[0]["id"], "\nEmbedding Length:", len(embedded_chunks[0]["embedding"]))

Generated embeddings for 25 chunks.
Example: chunk_0 
Embedding Length: 1024


In [21]:
embedded_chunks

[{'id': 'chunk_0',
  'text': 'Account Branch: HARI KE PATTAN\nMR JUGDISH SINGH | Address: HDFC BANK LTD\nKAWATAR NO.1   PATTI HARIKE TARN | PATTI ROAD, HARI KE PATTAN,\nTARAN,. | DIST - TARAN TARAN,\nHARI KE PATTAN 143412 | City: HARI KE PATTAN 143412\nPUNJAB INDIA | State:PUNJAB\nPhone no.: 18002600/18001600\nEmail: JAGDISHSINGH1469@GMAIL.COM\nJOINT HOLDERS: | OD Limit: 0 Currency: INR\n  -   | Cust ID: 235551811\n  -  \n  -  \n  -  \n  -  \nNomination: Registered | Account No: 50100618438641 - JUGDISH SINGH\nStatement From: 01/06/2024         To: 19/02/2025 | A/C Open Date: 17/05/2023\nAccount Status: Regular\nRTGS/NEFT IFSC: HDFC0002959 MICR: 143240306\n',
  'embedding': [0.033147182,
   -0.0086954925,
   0.014488238,
   -0.033233125,
   -0.021752536,
   0.037638016,
   0.08578042,
   -0.10958042,
   0.0100047635,
   -0.04490239,
   -0.038860682,
   0.031111129,
   -0.06734474,
   0.07698128,
   -0.12237842,
   -0.056547686,
   0.0047287163,
   0.0032529167,
   0.055153564,
   0.082

In [23]:
from fastembed import SparseTextEmbedding

# --- Initialize sparse embedding model ---
sparse_model = SparseTextEmbedding(model_name="Qdrant/bm25")

# --- Compute sparse embeddings for chunk texts ---
sparse_results = list(sparse_model.embed(chunk_texts))

# --- Structure output with metadata ---
sparse_embedded_chunks = []
for i, sparse_vec in enumerate(sparse_results):
    sparse_embedded_chunks.append({
        "id": chunks_for_qdrant[i]["id"],
        "text": chunks_for_qdrant[i]["text"],
        "sparse_embedding": {
            "indices": sparse_vec.indices,
            "values": sparse_vec.values
        },
        "metadata": chunks_for_qdrant[i]["metadata"]
    })

# --- Optional: Inspect a sample ---
print(f"Generated sparse embeddings for {len(sparse_embedded_chunks)} chunks.")
print("Sample ID:", sparse_embedded_chunks[0]["id"])
print("Indices:", sparse_embedded_chunks[0]["sparse_embedding"]["indices"])
print("Values:", sparse_embedded_chunks[0]["sparse_embedding"]["values"])

Generated sparse embeddings for 25 chunks.
Sample ID: chunk_0
Indices: [2071720296 1417432007 1312058989 1904930356 1023346985  738416413
 1228327439  536249406 2132027491  894871999  715590789 1558352984
  113102269 1810453357 1423306914 1100321665 1759695151  906280446
  694334040 1193939294  926955616 1966288579 1916928705 1844140545
  722829366 1942502812  755351349 1938865362  557666404 2037725392
  683003846 1746685302 1013647041 1840303211 1148371578  520409122
  764297089 1506681830 1726226603 1553598916 1737061407 1740864731
 1790667262  618706776 1421925395 1715992043 1642882560  402908866
 1555540962 1520550099 1137353851   35433555  516762017 2091876921
  718402331 1852771076 1299242728  423733604  900543307 1583324998
  627191273 1472368079  656599388 1190496381 2041556258 1603522155]
Values: [1.8357236  1.37904016 1.9149949  1.9149949  1.9149949  1.37904016
 1.69536424 1.69536424 1.37904016 1.37904016 1.37904016 1.37904016
 1.37904016 1.37904016 1.69536424 1.37904016 1.37

In [24]:
from qdrant_client import QdrantClient, models

client = QdrantClient(host="192.168.1.13", port=6333)

collection_name = "bank_statements_new"

try:
    client.get_collection(collection_name)
    print(f"Collection '{collection_name}' already exists.")
except Exception:
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config={
            "dense": models.VectorParams(
                size=1024,
                distance=models.Distance.COSINE,
            ),
        },
        sparse_vectors_config={
            "sparse": models.SparseVectorParams()
        },
        shard_number=1,
    )
    print(f"Collection '{collection_name}' created.")

Collection 'bank_statements_new' created.


C:\Users\prask\AppData\Local\Temp\ipykernel_26788\654238941.py:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [26]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, SparseVector
import uuid

# === Qdrant Configuration ===
QDRANT_URL = "http://192.168.1.13:6333/"
COLLECTION_NAME = "bank_statements_new"

# === Connect to Qdrant ===
client = QdrantClient(url=QDRANT_URL, check_compatibility=False)

# === Prepare Points ===
points = []
for i in range(len(embedded_chunks)):
    dense_vector = embedded_chunks[i]["embedding"]
    sparse_vec = sparse_embedded_chunks[i]["sparse_embedding"]
    text = embedded_chunks[i]["text"]
    metadata = embedded_chunks[i]["metadata"]

    point = PointStruct(
        id=str(uuid.uuid4()),
        vector={
            "dense": dense_vector,
            "sparse": SparseVector(
                indices=sparse_vec["indices"],
                values=sparse_vec["values"]
            )
        },
        payload={
            "text": text,
            **metadata
        }
    )
    points.append(point)

# === Upsert into Qdrant ===
client.upsert(
    collection_name=COLLECTION_NAME,
    points=points
)

print(f"Successfully inserted {len(points)} points into Qdrant collection '{COLLECTION_NAME}'.")

Successfully inserted 25 points into Qdrant collection 'bank_statements_new'.


In [21]:
from qdrant_client import QdrantClient

# Connect to your Qdrant instance
client = QdrantClient(url="http://192.168.1.13:6333/")

# Specify your collection and field details
collection_name = "bank_statements_new"            # Replace with your collection name
field_name = "transaction_date"              # Replace with the field you want to index

# Create a payload index for the field (as a keyword)
client.create_payload_index(
    collection_name=collection_name,
    field_name=field_name,
    field_schema="keyword"
)

print(f"Payload index for field '{field_name}' created in collection '{collection_name}'.")

Payload index for field 'transaction_date' created in collection 'bank_statements_new'.
